In [4]:
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [49]:
def new_box(box):
    l_d = [box[:,0].min(), box[:,1].min()]
    l_u = [box[:,0].min(), box[:,1].max()]
    r_d = [box[:,0].max(), box[:,1].min()]
    r_u = [box[:,0].max(), box[:,1].max()]
    return np.array([l_d, l_u, r_u, r_d])
def make_sample(img_name, img_m_name, new_name):
    hsv_min = np.array((2, 0, 0), np.uint8)
    hsv_max = np.array((255, 255, 255), np.uint8)
    img = cv2.imread(img_name)
    img_m = cv2.imread(img_m_name)
    hsv = cv2.cvtColor( img_m, cv2.COLOR_BGR2HSV )
    thresh = cv2.inRange( hsv, hsv_min, hsv_max )
    contours0, hierarchy = cv2.findContours( thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    result = pd.DataFrame()
    for cnt in contours0:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        n_box = new_box(box)
        area = int(rect[1][0]*rect[1][1])
        
        if area > 200 and area < 300000:
            cv2.drawContours(img,[n_box],0,(0,0,255),1)
            result = result.append(pd.DataFrame({
                'name':[new_name],
                'left_d x':[n_box[0,0]],
                'left_d y':[n_box[0,1]],
                'right_u x':[n_box[2,0]],
                'right_u y':[n_box[2,1]]
            }))

    return img, result
def all_img(image_names):
    result_log = pd.DataFrame()
    for i in range(image_names.shape[0]):
        img_name, img_m_name = image_names.iloc[i]['img name'], image_names.iloc[i]['img m name']
        img, r = make_sample(img_name, img_m_name, 'IMG'+str(i)+'.jpg')
        cv2.imwrite('IMG'+str(i)+'.jpg', img)
        result_log = result_log.append(r)
    result_log.set_index('name').to_csv('log.csv')
def get_img_names(f, t):
    result = pd.DataFrame()
    for i in range(f, t + 1):
        result = result.append(pd.DataFrame({
            'img name':[str(i)+'_1.jpg'],
            'img m name':[str(i)+'.jpg']
        }))
    return result

In [50]:
names = get_img_names(1, 4)

In [51]:
all_img(names)